# Efficient Usage of Different Models

The following code suppresses warning messages to ensure the output is clean and focused on the essential results.

In [ ]:
import warnings
import os
warnings.filterwarnings('ignore')
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'

Imports essential modules from CrewAI and initializes two tools, one for web scraping and one for search engine interaction. Also, sets up environment variables to securely store API keys required for accessing external services.

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from crewai import Agent, Task
import os
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Create a search tool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

import os
os.environ["OPENAI_API_KEY"] = 'ADD YOUR OPENAI API KEY HERE'
os.environ["SERPER_API_KEY"] = 'ADD YOUR SERPER API KEY HERE'
os.environ["GOOGLE_API_KEY"] = 'ADD YOUR GOOGLE API KEY HERE'

Initialize different models for different agents.

In [42]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from crewai import Agent

## call the gemini models
gemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                           verbose=True,
                           temperature=0.5,
                           google_api_key=os.getenv("GOOGLE_API_KEY"))
# Initialize the GPT-4 model using ChatOpenAI
gpt=ChatOpenAI(model="gpt-4o-2024-08-06",
               verbose=True,
               temperature=0.5,
               openai_api_key=os.getenv("OPENAI_API_KEY"))

Defines two agents, each leveraging a different LLM.

In [44]:
# Data Researcher Agent using Gemini and SerperSearch
article_researcher=Agent(
    role="Senior Researcher",
    goal='Unccover ground breaking technologies in {topic}',
    verbose=True,
    memory=True,
    backstory=(
        "Driven by curiosity, you're at the forefront of"
        "innovation, eager to explore and share knowledge that could change"
        "the world."
    ),
    tools=[search_tool],
    llm=gemini,
    allow_delegation=True

)

# Article Writer Agent using GPT
article_writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  llm=gpt,
  allow_delegation=False
)

Defining the tasks

In [45]:
# Research Task
research_task = Task(
    description=(
        "Conduct a thorough analysis on the given {topic}."
        "Utilize SerperSearch for any necessary online research. "
        "Summarize key findings in a detailed report."
    ),
    expected_output='A detailed report on the data analysis with key insights.',
    tools=[search_tool],
    agent=article_researcher,
)

# Writing Task
writing_task = Task(
    description=(
        "Write an insightful article based on the data analysis report. "
        "The article should be clear, engaging, and easy to understand."
    ),
    expected_output='A 6-paragraph article summarizing the data insights.',
    agent=article_writer,
)

Creates a crew and then kicks off the project.

In [40]:
from crewai import Crew, Process

# Form the crew and define the process
crew = Crew(
    agents=[article_researcher, article_writer],
    tasks=[research_task, writing_task],
    process=Process.sequential
)

research_inputs = {
    'topic': 'The rise in global tempratures from 2018 onwards'
}

# Kick off the crew
result = crew.kickoff(inputs=research_inputs)



> Entering new CrewAgentExecutor chain...
Thought: I need to gather information about the rise in global temperatures from 2018 onwards. I'll start by searching for data and research on this topic.

Action: Search the internet
Action Input: {"search_query": "global temperature rise 2018 onwards data"}
 


Search results: Title: Climate Change: Global Temperature
Link: http://www.climate.gov/news-features/understanding-climate/climate-change-global-temperature
Snippet: Earth's temperature has risen by an average of 0.11° Fahrenheit (0.06° Celsius) per decade since 1850, or about 2° F in total. The rate of warming since 1982 is ...
---
Title: 2018 fourth warmest year in continued warming trend, according to ...
Link: https://science.nasa.gov/earth/climate-change/2018-fourth-warmest-year-in-continued-warming-trend-according-to-nasa-noaa/
Snippet: Global temperatures in 2018 were 1.5 degrees Fahrenheit (0.83 degrees Celsius) warmer than the 1951 to 1980 mean, according to scientists at N

Converts the crew’s output into a Markdown string for better readability and displays the results in a formatted manner.

In [41]:
from IPython.display import Markdown

# Convert the CrewOutput object to a Markdown string
result_markdown = result.raw

# Display the result as Markdown
Markdown(result_markdown)

The rise in global temperatures from 2018 onwards has been a significant concern for scientists and policymakers worldwide. According to data from NASA and NOAA, 2018 marked the fourth warmest year on record, underscoring a persistent trend of increasing global temperatures. The warming trend has been particularly pronounced since 1982, with an average temperature increase of 0.11° Fahrenheit (0.06° Celsius) per decade. This rapid warming culminated in 2023, which was confirmed as the hottest year on record with a global average temperature of 14.98°C, surpassing the previous record set in 2016 by 0.17°C. April 2024 also set a new record for the hottest April, with an average surface air temperature significantly above the 1991-2020 average.

The record-breaking years of 2018, 2023, and April 2024 highlight the alarming pace at which global temperatures are rising. The data indicates that 2023 was not just slightly warmer but exceeded past records by a significant margin. This pattern of record-breaking temperatures is a stark reminder of the ongoing impact of climate change, driven by human activities. The warming trend since 1982 is notably higher than the historical average since 1850, illustrating the acceleration in temperature increases over recent decades.

The primary driver of these rising temperatures is the increasing concentration of greenhouse gases in the atmosphere, largely due to human activities such as burning fossil fuels. The impacts of this warming are already being felt across the globe, manifesting in more frequent and intense heat waves, rising sea levels, and extreme weather events like hurricanes, floods, and droughts. These changes are also affecting plant and animal life, as well as human health and well-being, posing significant challenges for communities worldwide.

In conclusion, the data clearly shows a steady rise in global temperatures since 2018, with 2023 marking a particularly alarming peak. This trend is a continuation of a long-term pattern of warming, primarily driven by human activities. The impacts of this warming are profound and are expected to become more severe in the future. Addressing this challenge requires urgent and concerted efforts to reduce greenhouse gas emissions and mitigate the impacts of climate change, highlighting the critical need for global cooperation and action.